In [1]:
from utils.image_paths import get_image_paths, get_new_folders
from service.colpali_server import ColPaliService
from pymilvus import MilvusClient
from service.milvus_retriever import MilvusColbertRetriever

model_dir = "/home/linux/yyj/colpali/colpali-v1.2-hf"  # 替换为实际的模型名称
device = "cuda:0"
milvus_uri = "http://localhost:19530"  # 替换为实际的Milvus服务器地址
collection_name = "colpali_hf"
image_dir = "./test_pages"  # 替换为实际的图片目录
manifest_file = "./manifest.json"

# 加载模型
colpali_service = ColPaliService(model_dir=model_dir, device=device)



/home/linux/anaconda3/envs/colpali/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.18s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
# 处理图片
new_folders, _, previous_number = get_new_folders(image_dir, manifest_file)
image_paths = get_image_paths(new_folders)
ds = colpali_service.process_images(image_paths)

# 初始化Milvus客户端和检索器
client = MilvusClient(uri=milvus_uri)
retriever = MilvusColbertRetriever(collection_name=collection_name, milvus_client=client)

# 创建集合和索引
if not client.has_collection(collection_name=collection_name):
    retriever.create_collection()
    retriever.create_index()

# 插入数据
for i in range(len(image_paths)):
    doc_id = i + previous_number
    data = {
        "colbert_vecs": ds[i].float().cpu().numpy(),
        "doc_id": doc_id,
        "filepath": image_paths[i],
    }
    retriever.insert(data)

Find 45 files in directory './test_pages' and its subdirectories


  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:20<00:00,  1.16s/it]


In [9]:
queries = [
    "阿姆斯特丹5月气温怎么样",
    "阿姆斯特丹12月降水量怎么样",
]
qs = colpali_service.process_queries(queries)

In [10]:
print(len(qs))
qs[0].shape

2


torch.Size([23, 128])

In [11]:
for query in qs:
    query = query.float().cpu().numpy()
    result = retriever.search(query, topk=1)
    print(result[0][1])

4
4
